### Problem
Consider a transport equation 
$$
2u_t+3u_x=0
$$
with boundary condition
$$
u(0,x) = f(x) = e^{-cos(x)}.
$$
The analytical solution is given by
$$
u(t,x) = f(x-\frac{3}{2}t) = e^{-cos(x-\frac{3}{2}t)}.
$$

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np